In [1]:
# docker pull emacski/tensorflow-serving:latest-linux_arm64
# docker run -t --rm -p 8501:8501 --mount type=bind,source=/tmp/model_name/,target=/models/model_name/ -e MODEL_NAME=model_name emacski/tensorflow-serving:latest-linux_arm64

In [15]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



In [33]:
#!docker pull tensorflow/serving

In [95]:
# esse funciona para quem n tem m1
#!docker pull tensorflow/serving:latest-gpu

In [22]:
# conda install tensorflow

In [34]:
import tensorflow as tf

In [35]:
# can also use mnist instead of vgg
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

In [36]:
model = VGG16(weights='imagenet')

In [22]:
pwd

'/Users/csamp/Documents/stack_ab'

In [37]:
img_path = 'tf_serving/lion.jpg'

img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)

print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 0s 224ms/step
Predicted: [('n04548280', 'wall_clock', 0.31570774), ('n02708093', 'analog_clock', 0.042893667), ('n04116512', 'rubber_eraser', 0.020840228)]


In [24]:
import time
current_time = int(time.time())
path = f"vgg16/{current_time}"
model.save(path)

INFO:tensorflow:Assets written to: vgg16/1669257063/assets


INFO:tensorflow:Assets written to: vgg16/1669257063/assets


In [96]:
path

'vgg16/1669257063'

In [3]:
# # how the folder should look like
# ├── vgg16
# │ ├── 1618397447
# │ │ ├── assets
# │ │ ├── saved_model.pb
# │ │ └── variables

In [26]:
# esse funciona
#!docker pull emacski/tensorflow-serving:latest-linux_arm64

latest-linux_arm64: Pulling from emacski/tensorflow-serving
Digest: sha256:37ae953d10a10f520fc7aca068c866f28a424894c47b1d06c8cec22e631e29be
Status: Image is up to date for emacski/tensorflow-serving:latest-linux_arm64
docker.io/emacski/tensorflow-serving:latest-linux_arm64


In [94]:
# !docker run -t --rm -p 8501:8501 \
#     -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
#     -e MODEL_NAME=vgg16 \
#     emacski/tensorflow-serving:latest-linux_arm6

In [50]:
#!docker run -p 8501:8501 --name tfserving_vgg16 --mount type=bind,source=vgg16/,target=vgg16/ -e MODEL_NAME=vgg16 -t tensorflow/serving

In [87]:
#!docker run -p 8502:8502 --name ttfserving_vgg -v source=/Users/csamp/Documents/stack_ab/vgg16/,target=vgg16 -e MODEL_NAME=vgg16 -t emacski/tensorflow-serving:latest-linux_arm64

In [88]:
pwd

'/Users/csamp/Documents/stack_ab'

In [108]:
#docker pull tensorflow/serving
# esse funciona
!docker run --rm -p 8501:8501 \
        --name tfserving_vgg16 \
        -v "/Users/csamp/Documents/stack_ab/vgg16:/models/vgg16" \
        -e MODEL_NAME=vgg16 \
        emacski/tensorflow-serving:latest-linux_arm64

2022-11-24 05:05:22.258051: I external/tf_serving/tensorflow_serving/model_servers/server.cc:89] Building single TensorFlow model file config:  model_name: vgg16 model_base_path: /models/vgg16
2022-11-24 05:05:22.259005: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2022-11-24 05:05:22.259020: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:591]  (Re-)adding model: vgg16
2022-11-24 05:05:22.369445: I external/tf_serving/tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: vgg16 version: 1669257063}
2022-11-24 05:05:22.369475: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: vgg16 version: 1669257063}
2022-11-24 05:05:22.369490: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: vgg16 version: 1669257063}
2022-11-24 05:05:22.370030: I external/org_tensorfl

In [109]:
# API endpoint can be accessed at http://localhost:8501. 

In [105]:
import json
import requests

def run_prediction(data,headers,endpoint):
    json_response = requests.post(endpoint,data=data,headers=headers)
    prediction = json.loads(json_response.text)
    return (decode_predictions(np.array(predictions['predictions']), top=3)[0])

In [110]:
endpoint = 'http://localhost:8501/v1/models/vgg16:predict'
data = json.dumps({"instances": x.tolist()})
headers = {"content-type": "application/json"}
run_prediction(data,headers,endpoint)

[('n04548280', 'wall_clock', 0.315708756),
 ('n02708093', 'analog_clock', 0.0428933091),
 ('n04116512', 'rubber_eraser', 0.0208401065)]

In [103]:
# code from https://cnvrg.io/how-to-serve-a-model-with-tensorflow/
# another way: https://medium.com/analytics-vidhya/deploy-production-ready-keras-model-with-tensorflow-serving-and-docker-85615245a9ed

In [104]:
json_response = requests.post('http://localhost:8501/v1/models/vgg16:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n04548280', 'wall_clock', 0.31570774), ('n02708093', 'analog_clock', 0.042893667), ('n04116512', 'rubber_eraser', 0.020840228)]


In [59]:
# ran docker with
# docker run -t --rm -p 8501:8501 \
#     -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
#     -e MODEL_NAME=vgg16 \
#     emacski/tensorflow-serving:latest-linux_arm64

I ran docker with:

`
docker run -t --rm -p 8501:8501 \
    -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
    -e MODEL_NAME=vgg16 \
    emacski/tensorflow-serving:latest-linux_arm6`
`

docker run -t --rm -p 8501:8501 \ 
    --mount type=bind,source=/tmp/model_name/,target=/models/model_name/ 
    -e MODEL_NAME=model_name \ 
    emacski/tensorflow-serving:latest-linux_arm64

In [55]:
!docker run -t --rm -p 8501:8501 \
    -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
    -e MODEL_NAME=vgg16 \
    emacski/tensorflow-serving:latest-linux_arm6

Unable to find image 'emacski/tensorflow-serving:latest-linux_arm6' locally
docker: Error response from daemon: manifest for emacski/tensorflow-serving:latest-linux_arm6 not found: manifest unknown: manifest unknown.
See 'docker run --help'.
